<a href="https://colab.research.google.com/github/GeorgeSherif/NLP-ChatEGP/blob/main/preprocessing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers sentencepiece

In [7]:
import nltk
import spacy 
import pandas as pd

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
df = pd.read_csv("/content/gdrive/MyDrive/NLP/all-data.csv",engine="python",encoding = "ISO-8859-1")

In [56]:
headerList = ['label', 'sentence']
  
# converting data frame to csv
df.to_csv("/content/gdrive/MyDrive/NLP/all-data.csv", header=headerList, index=False)

In [57]:
file2 = pd.read_csv("/content/gdrive/MyDrive/NLP/all-data.csv")

In [58]:
file2.head()

,label,sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [59]:
len(file2)

4846

In [51]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Bhuvana/t5-base-spellchecker")

model = AutoModelForSeq2SeqLM.from_pretrained("Bhuvana/t5-base-spellchecker")


def correct(inputs):
    input_ids = tokenizer.encode(inputs,return_tensors='pt')
    sample_output = model.generate(
        input_ids,
        do_sample=True,
        max_length=50,
        top_p=0.99,
        num_return_sequences=1
    )
    res = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    return res

In [29]:
from transformers import pipeline

fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

In [33]:
print(fix_spelling("Nevertheless , the development can not be allowed to ruin the print newspaper , which continues to be Sanoma News ' main medium .".replace('`', '') ,max_length=2048))


[{'generated_text': "Nevertheless  the development can not be allowed to ruin the print newspaper , which continues to be Sanoma News ' main medium ."}]


In [54]:
corrected = []
for i in range(len(file2)):
   correct = correct(file2['sentence'][i])
   corrected.append(correct)
   print(i)
   print(correct)
   print(file2['sentence'][i])

TypeError: ignored

In [ ]:
corrected

In [60]:
tokens = []
for i in range(len(file2)):
  tokens.append(nltk.word_tokenize(file2['sentence'][i]))

In [61]:
file2['token'] = tokens

In [62]:
file2.head()

,label,sentence,token
0,neutral,"According to Gran , the company has no plans t...","[According, to, Gran, ,, the, company, has, no..."
1,neutral,Technopolis plans to develop in stages an area...,"[Technopolis, plans, to, develop, in, stages, ..."
2,negative,The international electronic industry company ...,"[The, international, electronic, industry, com..."
3,positive,With the new production plant the company woul...,"[With, the, new, production, plant, the, compa..."
4,positive,According to the company 's updated strategy f...,"[According, to, the, company, 's, updated, str..."


In [ ]:
p_stemmer = PorterStemmer()
stemmed = []

for i in range(len(file2)):
  nltk_tokenList = word_tokenize(file2['sentence'][i]) 
  nltk_stemedList = []
  for word in nltk_tokenList:
     nltk_stemedList.append(p_stemmer.stem(word))
  stemmed.append(nltk_stemedList)

In [ ]:
file2['stem'] = stemmed

In [63]:
nltk_stop_words = set(stopwords.words('english'))
stopRemoved = []
# print(nltk_stop_words)
for i in range(len(file2)):
  filtered_sentence =[]
  for w in file2['token'][i]:  
    if w not in nltk_stop_words:  
        filtered_sentence.append(w)  
  stopRemoved.append(filtered_sentence)

In [64]:
file2['stop'] = stopRemoved

In [65]:
punctRem = []
punctuations="?:!.,;"
for i in range(len(file2)):
   removedPuncForSentence = []
   for word in file2['stop'][i]:
      if word not in punctuations:
          removedPuncForSentence.append(word)
   punctRem.append(removedPuncForSentence)

In [66]:
file2['punct'] = punctRem

In [67]:
print(file2['sentence'][0])

According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .


In [ ]:
print(file2['punct'][0])

['technopoli', 'plan', 'develop', 'stage', 'area', 'less', '100,000', 'squar', 'meter', 'order', 'host', 'compani', 'work', 'comput', 'technolog', 'telecommun', 'statement', 'said']


In [68]:
finalizedForm = []
for i in range(len(file2)):
  concatStr = ' '
  for j in range(len(file2['punct'][i])):
      concatStr += ' ' + file2['punct'][i][j]
  finalizedForm.append(concatStr)

In [69]:
file2['bertForm'] = finalizedForm

In [71]:
file_path = '/content/gdrive/MyDrive/NLP/withoutStemming.csv'
file2.to_csv(file_path, index=False)